<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/08_RAG_Avanzado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Avanzado con IMDB: Técnicas Modernas de Recuperación y Generación

Este notebook implementa un sistema RAG avanzado que incluye:
1. Re-ranking de resultados
2. Generación aumentada con múltiples contextos
3. Personalización de documentos
4. Uso de plantillas para prompts
5. Fusión inteligente de contextos

Usaremos un subconjunto de IMDB para demostrar estas técnicas.

In [1]:
!pip install datasets
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
from typing import List, Dict
import json
from dataclasses import dataclass
from tqdm.auto import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# 1. PREPARACIÓN DE DATOS

Utilizamos el dataset MovieLens (versión reducida).
Puedes descargar el original de: https://grouplens.org/datasets/movielens/
- Versión small (100k ratings): https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
- Versión latest (más completa): https://files.grouplens.org/datasets/movielens/ml-latest.zip

El dataset incluye varios archivos:
- movies.csv: información básica de películas (id, título, géneros)
- ratings.csv: ratings de usuarios
- links.csv: links a IMDb y TMDb
- tags.csv: tags asignados por usuarios

In [2]:
print("Cargando datos...")

# Cargar películas
movies_df = pd.read_csv("https://raw.githubusercontent.com/cbadenes/curso-pln/main/datasets/movielens/movies.csv")
print(f"Películas cargadas: {len(movies_df)}")

# Cargar y procesar ratings
ratings_df = pd.read_csv("https://raw.githubusercontent.com/cbadenes/curso-pln/main/datasets/movielens/ratings.csv")
print(f"Ratings cargados: {len(ratings_df)}")

# Calcular rating promedio por película
avg_ratings = ratings_df.groupby('movieId')['rating'].agg(['mean', 'count']).reset_index()
avg_ratings.columns = ['movieId', 'rating', 'num_ratings']

# Unir datos y rellenar valores nulos
df = movies_df.merge(avg_ratings, on='movieId', how='left')
df['rating'] = df['rating'].fillna(0)
df['num_ratings'] = df['num_ratings'].fillna(0)

# Crear descripciones simples
def create_description(row):
    description = f"La película {row['title']} es de {row['genres'].replace('|', ', ')}"
    if row['num_ratings'] > 0:
        description += f". Tiene una valoración de {row['rating']:.1f} con {int(row['num_ratings'])} votos"
    return description + "."

df['description'] = df.apply(create_description, axis=1)
print("\nDatos preparados.")

Cargando datos...
Películas cargadas: 9742
Ratings cargados: 100836

Datos preparados.


# 2. BÚSQUEDA SEMÁNTICA CON RE-RANKING

In [ ]:
print("\nPreparando modelos...")

# Modelos
retriever = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Crear embeddings
print("Generando embeddings...")
embeddings = retriever.encode(df['description'].values, show_progress_bar=True)

def search_movies(query, top_k=5):
    """Búsqueda inicial de películas."""
    query_embedding = retriever.encode(query)
    similarities = np.dot(embeddings, query_embedding)
    top_idxs = np.argsort(similarities)[::-1][:top_k]
    return df.iloc[top_idxs]

def rerank_results(query, results):
    """Re-ranking de resultados."""
    pairs = [[query, row['description']] for _, row in results.iterrows()]
    scores = reranker.predict(pairs)
    results = results.copy()
    results['score'] = scores
    return results.sort_values('score', ascending=False)


Preparando modelos...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Generando embeddings...


Batches:   0%|          | 0/305 [00:00<?, ?it/s]

# 3. PERSONALIZACIÓN Y FUSIÓN DE CONTEXTOS

In [ ]:
class UserProfile:
    """Perfil de usuario para personalización."""
    def __init__(self, favorite_genres, preferred_years=None, min_rating=None):
        self.favorite_genres = favorite_genres
        self.preferred_years = preferred_years or []
        self.min_rating = min_rating or 0

def personalize_results(results, profile):
    """Personaliza resultados según perfil de usuario."""
    results = results.copy()

    # Boost por géneros favoritos
    results['boost'] = results['genres'].apply(
        lambda x: sum(genre in x for genre in profile.favorite_genres)
    )

    # Boost por rating mínimo
    if profile.min_rating > 0:
        results['boost'] += (results['rating'] >= profile.min_rating).astype(int)

    # Ajustar score final
    results['score'] = results['score'] * (1 + 0.2 * results['boost'])

    return results.sort_values('score', ascending=False)

def fuse_contexts(results, max_tokens=1000):
    """Fusiona múltiples resultados en un contexto coherente."""
    context = []
    current_tokens = 0

    for _, movie in results.iterrows():
        # Crear snippet informativo
        snippet = (
            f"• {movie['title']}\n"
            f"  Géneros: {movie['genres'].replace('|', ', ')}\n"
            f"  Rating: {movie['rating']:.1f}/5 ({int(movie['num_ratings'])} votos)\n"
        )

        # Añadir si hay espacio (aproximación simple de tokens)
        if current_tokens + len(snippet.split()) > max_tokens:
            break

        context.append(snippet)
        current_tokens += len(snippet.split())

    return "\n".join(context)

# 4. PLANTILLAS PARA PROMPTS

In [ ]:
class PromptTemplates:
    """Plantillas para diferentes tipos de consultas."""

    @staticmethod
    def movie_recommendation(query, context, profile=None):
        """Plantilla para recomendación de películas."""
        prompt = f"""
        Consulta del usuario: {query}

        Información relevante sobre películas disponibles:
        {context}
        """

        if profile:
            prompt += f"""
            Perfil del usuario:
            - Géneros favoritos: {', '.join(profile.favorite_genres)}
            - Rating mínimo deseado: {profile.min_rating}
            """

        prompt += """
        Por favor, genera una respuesta que:
        1. Recomiende las películas más relevantes
        2. Explique por qué son adecuadas para la consulta
        3. Mencione los géneros y valoraciones
        4. Sugiera películas similares si las hay
        """

        return prompt

# 5. EJEMPLO DE USO COMPLETO

In [ ]:
print("\nEjemplo de búsqueda personalizada:")

# Definir perfil de usuario
user_profile = UserProfile(
    favorite_genres=['Action', 'Sci-Fi'],
    min_rating=4.0
)

# Consulta de ejemplo
query = "películas de acción con efectos especiales"
print(f"\nBuscando: '{query}'")

# 1. Búsqueda inicial
initial_results = search_movies(query, top_k=10)

# 2. Re-ranking
reranked_results = rerank_results(query, initial_results)

# 3. Personalización
personalized_results = personalize_results(reranked_results, user_profile)

# 4. Fusión de contextos
context = fuse_contexts(personalized_results)

# 5. Generación de prompt
prompt = PromptTemplates.movie_recommendation(query, context, user_profile)

# Mostrar resultados
print("\nResultados finales:")
for _, movie in personalized_results.head().iterrows():
    print(f"\n- {movie['title']}")
    print(f"  Géneros: {movie['genres'].replace('|', ', ')}")
    print(f"  Rating: {movie['rating']:.1f}/5 ({int(movie['num_ratings'])} votos)")
    print(f"  Score final: {movie['score']:.3f}")

print("\nContexto fusionado para LLM:")
print(context)

print("\nPrompt generado:")
print(prompt)

# EXPLICACIÓN DE TÉCNICAS IMPLEMENTADAS:

1. Búsqueda y Re-ranking:
   - Búsqueda inicial rápida con embeddings
   - Re-ranking preciso con cross-encoder
   - Balance entre velocidad y precisión

2. Personalización:
   - Perfil de usuario con preferencias
   - Boost por géneros favoritos
   - Filtrado por rating mínimo
   - Scores ajustados según preferencias

3. Fusión de Contextos:
   - Selección inteligente de información
   - Control de longitud para el LLM
   - Formato estructurado y claro

4. Plantillas de Prompts:
   - Estructura clara y consistente
   - Inclusión de contexto relevante
   - Instrucciones específicas para el LLM
   - Adaptación al perfil del usuario

Ventajas de este enfoque:
- Resultados más relevantes y personalizados
- Mejor uso del contexto disponible
- Prompts más efectivos para el LLM
- Flexibilidad para diferentes casos de uso